# Computing the centroid
I want to be able to compute the centroid of raster while taking into account the valuesof the raster. In other words, I want the centre of mass of the raster. 

I found a way of doing this [here](https://stackoverflow.com/a/71459644) and below I explore why this works.

Make an example datarray

In [1]:
import xarray as xr
import numpy as np


In [2]:
A = xr.DataArray(np.array([[0,0,1,1,0,0],
                           [0,1,1,1,0,0],
                           [0,1,1,1,0,0],
                           [0,1,1,0,0,0],
                           [0,0,0,0,1,0]]),
                 dims=['y','x'],
                 coords={'x': [10,20,30,40,50,60],
                         'y': [50,40,30,20,10]})

In [3]:
A

<xarray.DataArray (y: 5, x: 6)>
array([[0, 0, 1, 1, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0]])
Coordinates:
  * x        (x) int64 10 20 30 40 50 60
  * y        (y) int64 50 40 30 20 10

One way of computing the center of mass of this array is to use a scipy function:

In [4]:
from scipy import ndimage
CM = ndimage.center_of_mass(A.values)
x_center = np.interp(CM[1], np.arange(0,len(A.x)), A.x) 
x_center

31.818181818181817

This has the disadvantage that it takes us out of xarray, risking getting mixed up about the orientation of the array.

This is the method that I found in the stackoverflow post. 

In [5]:
A.x.weighted(A).mean()

<xarray.DataArray 'x' ()>
array(31.81818182)

It gives the same answer as the scipy method, but initially i wasnt sure what it was doing because `A.x` is 1D while the array it is beind weighted by (`A`) is 2D. The answer is that `A.x` is broadcast to the shape of `A` before the weighted mean is computed. 

It makes sense that this would give us the x cooridianate of the centroid, and to make sure this is what is happening we can do the broadcasting manually and check we get the same answer:

In [6]:
# extrude x and y to 2D meshgrids
x, y = np.meshgrid(A.x.values, A.y.values)
# put the x-coordinate meshgrid into an xarray
x_mesh_xr = xr.DataArray(x, dims=['y','x'], coords={'x': A.x, 'y': A.y})
# and use A to weight the x meshgrid in the mean calculation. 
x_mesh_xr.weighted(A).mean()

<xarray.DataArray ()>
array(31.81818182)

This is the same answer, so I am confident this is what is happening with 
```
A.x.weighted(A).mean()
```

Next is to test how this approach with a 3D array.

In [34]:
temp = xr.tutorial.load_dataset('air_temperature')
A3d = temp.air.isel(time=slice(0,30))
A3d

<xarray.DataArray 'air' (time: 30, lat: 25, lon: 53)>
array([[[241.2    , 242.5    , 243.5    , ..., 232.79999, 235.5    ,
         238.59999],
        [243.79999, 244.5    , 244.7    , ..., 232.79999, 235.29999,
         239.29999],
        [250.     , 249.79999, 248.89   , ..., 233.2    , 236.39   ,
         241.7    ],
        ...,
        [296.6    , 296.19998, 296.4    , ..., 295.4    , 295.1    ,
         294.69998],
        [295.9    , 296.19998, 296.79   , ..., 295.9    , 295.9    ,
         295.19998],
        [296.29   , 296.79   , 297.1    , ..., 296.9    , 296.79   ,
         296.6    ]],

       [[242.09999, 242.7    , 243.09999, ..., 232.     , 233.59999,
         235.79999],
        [243.59999, 244.09999, 244.2    , ..., 231.     , 232.5    ,
         235.7    ],
        [253.2    , 252.89   , 252.09999, ..., 230.79999, 233.39   ,
         238.5    ],
...
        [297.     , 296.19998, 295.29   , ..., 295.69998, 295.1    ,
         294.     ],
        [297.1    , 296.6    , 295.79   , ..., 295.69998, 295.1    ,
         294.1    ],
        [297.1    , 296.9    , 296.69998, ..., 296.1    , 295.29   ,
         294.6    ]],

       [[252.7    , 253.09999, 253.29999, ..., 243.7    , 244.29999,
         245.89   ],
        [253.7    , 254.29999, 254.5    , ..., 248.89   , 250.09999,
         252.09999],
        [250.5    , 249.29999, 248.09999, ..., 249.39   , 252.79999,
         256.8    ],
        ...,
        [297.     , 296.     , 295.1    , ..., 295.4    , 295.1    ,
         294.     ],
        [297.19998, 296.69998, 296.19998, ..., 295.5    , 295.     ,
         294.     ],
        [297.5    , 296.9    , 297.     , ..., 295.79   , 295.1    ,
         294.5    ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-08T06:00:00
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

We want to do the center of mass calculation at every time step. It would be convenient if the following did this:

In [35]:
lon_center_xr = A3d.lon.weighted(A3d).mean(dim=['lat','lon'])
lon_center_xr

<xarray.DataArray 'lon' (time: 30)>
array([264.93762, 264.9429 , 264.98276, 264.98404, 264.96255, 264.94635,
       264.98004, 264.94083, 264.90518, 264.88922, 264.87344, 264.91693,
       264.8727 , 264.92392, 264.95444, 264.98413, 264.93732, 264.94333,
       264.963  , 265.00262, 264.9525 , 264.9325 , 264.92767, 264.91528,
       264.90726, 264.9022 , 264.93362, 264.97504, 264.98312, 265.04477],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2013-01-01 ... 2013-01-08T06:00:00

It seems to work, but to check, we can loop of the time dimension and compute the x-coord of the center of mass for each one:

In [51]:
lon_centers = []
times = A3d.time.values
for time in times:
    lon_centers.append(A3d.lon.weighted(A3d.sel(time=time)).mean(dim = ['lon', 'lat']).values)
# convert this to a numpy array
lon_centers = np.array(lon_centers)


This is the same as simply calling ```A3d.lon.weighted(A3d).mean(dim=['lat','lon'])```, so this appears to be working. 

In [53]:
all(lon_centers == lon_center_xr.values)

True